In [18]:
import pandas as pd
import numpy as np

df=pd.read_csv('train.csv')


In [19]:
#Data Engineering

df['Title']=df['Name'].str.extract(r'(Mr|Mrs|Miss|Dr|Master|Don|Rev|Ms|Major|Col|Capt|Mme|Mlle|Countess|Jonkheer)')
def GroupTitle(X):
    #if X['Title'] in ['Miss','Ms']:
       # y='Miss'
    if X['Title'] in ['Mme','Mlle','Countess','Don','Master','Major','Col','Capt','Jonkheer']:
        y='Nobility'
    elif X['Title'] in ['Dr','Rev']:
        y='Prof'
    else:
        y='Pleb' #X['Title']
    return y
 #Consider grouping Mr, Mrs, Miss all together too - any information would be held in sex and age       
df['GTitle']=df.apply(GroupTitle,axis=1)
df['FamilySize'] = df['Parch'] + df['SibSp'] + 1
df['Cabin']=df['Cabin'].to_string()
df['Cabin1']=df['Cabin'].map(lambda x: x[0])


In [20]:
from sklearn.preprocessing import OneHotEncoder, Imputer, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

y=df['Survived']
cat_attrib= ['Sex','Pclass','Embarked','GTitle','Cabin1']
num_attrib= ['Age','Fare','SibSp','Parch','FamilySize']
df['Embarked'].fillna('S',inplace=True)

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        return X[self.attribute_names].values


num_pipeline=Pipeline([('df_selector',DataFrameSelector(num_attrib)),
                       ('imputer', Imputer(strategy='mean')),('MinMax_scaler',MinMaxScaler())])
cat_pipeline=Pipeline([('selector',DataFrameSelector(cat_attrib)),
                       ('cat_encoder',OneHotEncoder())])
full_pipeline=FeatureUnion(transformer_list=[('num_pipe',num_pipeline),('cat_pipe',cat_pipeline)])
df_proc=full_pipeline.fit_transform(df)

log_clf = LogisticRegression()
svc_clf=SVC(kernel='poly',gamma='scale', probability = True)
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=10,algorithm='SAMME.R',
                             learning_rate=0.5)
rf_clf=RandomForestClassifier(n_estimators=30, max_features = 14,max_depth=14,min_samples_leaf=3,min_samples_split=9, bootstrap = True)
voting_clf=VotingClassifier(estimators=[('lr',log_clf),('rf',rf_clf),('svc',svc_clf),('ada',ada_clf)],voting='soft')
voting_clf.fit(df_proc,y)



/anaconda3/envs/Enviro_Tensor_Flow/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/anaconda3/envs/Enviro_Tensor_Flow/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)), ('rf', RandomFo...ne,
            splitter='best'),
          learning_rate=0.5, n_estimators=10, random_state=None))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [22]:
#Test Data

df_test=pd.read_csv('test.csv')
df_test['Title']=df_test['Name'].str.extract(r'(Mr|Mrs|Miss|Dr|Master|Don|Rev|Ms|Major|Col|Capt|Mme|Mlle|Countess|Jonkheer)')    
df_test['GTitle']=df_test.apply(GroupTitle,axis=1)
df_test['FamilySize'] = df_test['Parch'] + df_test['SibSp'] + 1
df_test['Cabin']=df_test['Cabin'].to_string()
df_test['Cabin1']=df_test['Cabin'].map(lambda x: x[0])

df_proc_test=full_pipeline.transform(df_test)
ID=pd.Series(df_test['PassengerId'],name='PassengerId')
test_Survived = pd.Series(voting_clf.predict(df_proc_test), name="Survived")
results = pd.concat([ID,test_Survived],axis=1)
results.to_csv("titanic_test_results_voting2.csv",index=False)
